# 2022 / 12 / 25

In [1]:
using IntervalArithmetic
using LaTeXStrings

In [2]:
include("./concatImportanceMethod/concatImportanceMethod.jl")
include("./display/concatImportanceMethod.jl")
include("./display/interval.jl")
include("./display/twofoldInterval.jl")
include("./twofoldIntervalPCM/twofoldIntervalPCM.jl")

intervalPCM2TwofoldIntervalPCM

In [3]:
function concatImportanceMethod(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    approximationResults = solveConcatImportanceMethodApproximationLP.([A₁, A₂])
    display(L"\text{各DMの上近似と下近似}")
    displayConcatImportanceMethodApproximationLPResults(approximationResults)

    tBoundaries = calculateConcatImportanceMethodTBoundaries.(approximationResults)
    display(L"\text{各DMの上近似と下近似の}t\text{の範囲}")
    displayConcatImportanceMethodTBoundaries(tBoundaries)

    result = solveConcatImportanceMethodConcatLP(approximationResults, tBoundaries)
    display(L"\text{LPの最適解}")
    displayConcatImportanceMethodConcatLPResult(result)

    Â = generateConcatImportanceMethodPCM(result)
    display(L"\text{結合後のPCM}")
    display(L"\hat{A} = %$(twofoldIntervalMatrixLaTeXString(Â))")
end

concatImportanceMethod (generic function with 1 method)

In [4]:
function getIntervalVector(
        approximationLPResult::ConcatImportanceMethodApproximationLPResult{T},
        concatLPResult::ConcatImportanceMethodConcatLPResult{T},
        k::Integer
        ) where {T <: Real}
    m = length(concatLPResult.t⁻)
    n = length(concatLPResult.wᴸ)

    wₖᴸ⁻ = approximationLPResult.wₖᴸ⁻
    wₖᵁ⁻ = approximationLPResult.wₖᵁ⁻
    wₖᴸ⁺ = approximationLPResult.wₖᴸ⁺
    wₖᵁ⁺ = approximationLPResult.wₖᵁ⁺

    tₖ⁻ = concatLPResult.t⁻[k]
    tₖ⁺ = concatLPResult.t⁺[k]

    tₖ⁺Wₖ⁺ = map(i -> tₖ⁺ * (wₖᴸ⁺[i]..wₖᵁ⁺[i]), 1:n)

    if any(isinf.(wₖᴸ⁻)) || any(isinf.(wₖᵁ⁻))
        return (tₖ⁻Wₖ⁻=fill(emptyinterval(), n), tₖ⁺Wₖ⁺=tₖ⁺Wₖ⁺)
    end

    tₖ⁻Wₖ⁻ = map(i -> tₖ⁻ * (wₖᴸ⁻[i]..wₖᵁ⁻[i]), 1:n)
    return (tₖ⁻Wₖ⁻=tₖ⁻Wₖ⁻, tₖ⁺Wₖ⁺=tₖ⁺Wₖ⁺)
end

getIntervalVector (generic function with 1 method)

In [5]:
function concatIntervalVectors(
        approximationLPResults::AbstractArray{ConcatImportanceMethodApproximationLPResult{T}, 1},
        concatLPResult::ConcatImportanceMethodConcatLPResult{T}
        ) where {T <: Real}
    m = length(concatLPResult.t⁻)
    n = length(concatLPResult.wᴸ)

    ⋂tₖ⁻Wₖ⁻ = fill(entireinterval(), n); ⋃tₖ⁻Wₖ⁻ = fill(emptyinterval(), n)
    ⋂tₖ⁺Wₖ⁺ = fill(entireinterval(), n); ⋃tₖ⁺Wₖ⁺ = fill(emptyinterval(), n)

    for k = 1:m
        intervalVectors = getIntervalVector(approximationLPResults[k], concatLPResult, k)
        for i = 1:n
            ⋂tₖ⁻Wₖ⁻[i] = ⋂tₖ⁻Wₖ⁻[i] ∩ intervalVectors.tₖ⁻Wₖ⁻[i]
            ⋃tₖ⁻Wₖ⁻[i] = ⋃tₖ⁻Wₖ⁻[i] ∪ intervalVectors.tₖ⁻Wₖ⁻[i]
            ⋂tₖ⁺Wₖ⁺[i] = ⋂tₖ⁺Wₖ⁺[i] ∩ intervalVectors.tₖ⁺Wₖ⁺[i]
            ⋃tₖ⁺Wₖ⁺[i] = ⋃tₖ⁺Wₖ⁺[i] ∪ intervalVectors.tₖ⁺Wₖ⁺[i]
        end
    end

    return (⋂tₖ⁻Wₖ⁻=⋂tₖ⁻Wₖ⁻, ⋃tₖ⁻Wₖ⁻=⋃tₖ⁻Wₖ⁻, ⋂tₖ⁺Wₖ⁺=⋂tₖ⁺Wₖ⁺, ⋃tₖ⁺Wₖ⁺=⋃tₖ⁺Wₖ⁺)
end

concatIntervalVectors (generic function with 1 method)

In [6]:
function displayConcatIntervalVectors(
        approximationLPResults::AbstractArray{ConcatImportanceMethodApproximationLPResult{T}, 1},
        concatLPResult::ConcatImportanceMethodConcatLPResult{T}
        ) where {T <: Real}
    intervalVectors = concatIntervalVectors(approximationLPResults, concatLPResult)

    display(L"""
        \bigcap_{k \in M^{'}} t_k^- W_k^- = %$(intervalVectorLaTeXString(intervalVectors.⋂tₖ⁻Wₖ⁻)), ~~
        \bigcup_{k \in M^{'}} t_k^- W_k^- = %$(intervalVectorLaTeXString(intervalVectors.⋃tₖ⁻Wₖ⁻)), ~~
        \bigcap_{k \in M} t_k^+ W_k^+ = %$(intervalVectorLaTeXString(intervalVectors.⋂tₖ⁺Wₖ⁺)), ~~
        \bigcup_{k \in M} t_k^+ W_k^+ = %$(intervalVectorLaTeXString(intervalVectors.⋃tₖ⁺Wₖ⁺))
    """)
end

displayConcatIntervalVectors (generic function with 1 method)

---

$\bigcap_{k \in M^{'}} t_k^+ W_k^-, \bigcup_{k \in M^{'}} t_k^- W_k^-$ と $t_k^- \left[ w_i^\text{L}, w_i^\text{U} \right]$ ，
$\bigcap_{k \in M^{'}} t_k^+ W_k^+, \bigcup_{k \in M^{'}} t_k^+ W_k^+$ と $t_k^+ \left[ w_i^\text{L}, w_i^\text{U} \right]$ の包含関係を調べる．

In [63]:
A₁ = [
    1..1 (1/3.3)..(1/3)
    3..3.3 1..1
]
A₂ = [
    1..1 (1/5)..(1/3.3)
    3.3..5 1..1
]

display(L"A_1 = %$(intervalMatrixLaTeXString(A₁))")
display(L"A_2 = %$(intervalMatrixLaTeXString(A₂))")

L"$A_1 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 0.303, 0.333 \right] \\ \left[ 3.0, 3.3 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

L"$A_2 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 0.2, 0.303 \right] \\ \left[ 3.3, 5.0 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

In [64]:
approximationResults = solveConcatImportanceMethodApproximationLP.([A₁, A₂])
displayConcatImportanceMethodApproximationLPResults(approximationResults)

L"$    w_{1}^{\text{L}-} = \begin{bmatrix} 0.25  \\  0.75 \end{bmatrix}, ~~
    w_{1}^{\text{U}-} = \begin{bmatrix} 0.25  \\  0.75 \end{bmatrix}, ~~
    w_{1}^{\text{L}+} = \begin{bmatrix} 0.233  \\  0.75 \end{bmatrix}, ~~
    w_{1}^{\text{U}+} = \begin{bmatrix} 0.25  \\  0.767 \end{bmatrix}
$"

L"$    w_{2}^{\text{L}-} = \begin{bmatrix} 0.233  \\  0.767 \end{bmatrix}, ~~
    w_{2}^{\text{U}-} = \begin{bmatrix} 0.233  \\  0.767 \end{bmatrix}, ~~
    w_{2}^{\text{L}+} = \begin{bmatrix} 0.167  \\  0.767 \end{bmatrix}, ~~
    w_{2}^{\text{U}+} = \begin{bmatrix} 0.233  \\  0.833 \end{bmatrix}
$"

In [65]:
tBoundaries = calculateConcatImportanceMethodTBoundaries.(approximationResults)
displayConcatImportanceMethodTBoundaries(tBoundaries)

L"$    t_{1}^{\text{L}-} = 1.0, ~~
    t_{1}^{\text{U}-} = 1.0, ~~
    t_{1}^{\text{L}+} = 1.0, ~~
    t_{1}^{\text{U}+} = 1.0
$"

L"$    t_{2}^{\text{L}-} = 1.0, ~~
    t_{2}^{\text{U}-} = 1.0, ~~
    t_{2}^{\text{L}+} = 1.0, ~~
    t_{2}^{\text{U}+} = 1.0
$"

In [66]:
result = solveConcatImportanceMethodConcatLP(approximationResults, tBoundaries)
displayConcatImportanceMethodConcatLPResult(result)

L"$t^- = \begin{bmatrix} 1.0  \\  1.0 \end{bmatrix}, ~~
t^+ = \begin{bmatrix} 1.0  \\  1.0 \end{bmatrix}
$"

L"$W = \begin{bmatrix}\left[ 0.233, 0.25 \right] \\ \left[ 0.75, 0.767 \right]\end{bmatrix}$"

L"$v^{\text{L}-} = \begin{bmatrix} 0.233  \\  0.75 \end{bmatrix}, ~~
v^{\text{U}-} = \begin{bmatrix} 0.25  \\  0.767 \end{bmatrix}, ~~
v^{\text{L}+} = \begin{bmatrix} 0.233  \\  0.767 \end{bmatrix}, ~~
v^{\text{U}+} = \begin{bmatrix} 0.233  \\  0.767 \end{bmatrix}
$"

L"$\varepsilon^\text{L} = \begin{bmatrix} 0.0  \\  0.017 \end{bmatrix}, ~~
\varepsilon^\text{U} = \begin{bmatrix} 0.017  \\  0.0 \end{bmatrix}
$"

In [67]:
displayConcatIntervalVectors(approximationResults, result)

L"$    \bigcap_{k \in M^{'}} t_k^- W_k^- = \begin{bmatrix} \emptyset \\ \emptyset \end{bmatrix}, ~~
    \bigcup_{k \in M^{'}} t_k^- W_k^- = \begin{bmatrix} \left[ 0.233, 0.25 \right] \\ \left[ 0.75, 0.767 \right] \end{bmatrix}, ~~
    \bigcap_{k \in M} t_k^+ W_k^+ = \begin{bmatrix} \left[ 0.233, 0.233 \right] \\ \left[ 0.767, 0.767 \right] \end{bmatrix}, ~~
    \bigcup_{k \in M} t_k^+ W_k^+ = \begin{bmatrix} \left[ 0.167, 0.25 \right] \\ \left[ 0.75, 0.833 \right] \end{bmatrix}
$"

---

順序関係がキャンセリングによって変化するかどうかのテスト

In [87]:
A₁ = [
    1..1 1..3 1..3.1
    (1/3)..1 1..1 1.1..1.2
    (1/3.1)..1 (1/1.2)..(1/1.1) 1..1
]
A₂ = [
    1..1 1..3 1..3.1
    (1/3)..1 1..1 1.1..1.2
    (1/3.1)..1 (1/1.2)..(1/1.1) 1..1
]

display(L"A_1 = %$(intervalMatrixLaTeXString(A₁))")
display(L"A_2 = %$(intervalMatrixLaTeXString(A₂))")

concatImportanceMethod(A₁, A₂)
print()

L"$A_1 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 1.0, 3.0 \right] & \left[ 1.0, 3.1 \right] \\ \left[ 0.333, 1.0 \right] & \left[ 1.0, 1.0 \right] & \left[ 1.1, 1.2 \right] \\ \left[ 0.323, 1.0 \right] & \left[ 0.833, 0.909 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

L"$A_2 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 1.0, 3.0 \right] & \left[ 1.0, 3.1 \right] \\ \left[ 0.333, 1.0 \right] & \left[ 1.0, 1.0 \right] & \left[ 1.1, 1.2 \right] \\ \left[ 0.323, 1.0 \right] & \left[ 0.833, 0.909 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

L"$\text{各DMの上近似と下近似}$"

L"$    w_{1}^{\text{L}-} = \begin{bmatrix} 0.591  \\  0.214  \\  0.195 \end{bmatrix}, ~~
    w_{1}^{\text{U}-} = \begin{bmatrix} 0.591  \\  0.214  \\  0.195 \end{bmatrix}, ~~
    w_{1}^{\text{L}+} = \begin{bmatrix} 0.333  \\  0.201  \\  0.195 \end{bmatrix}, ~~
    w_{1}^{\text{U}+} = \begin{bmatrix} 0.604  \\  0.333  \\  0.333 \end{bmatrix}
$"

L"$    w_{2}^{\text{L}-} = \begin{bmatrix} 0.591  \\  0.214  \\  0.195 \end{bmatrix}, ~~
    w_{2}^{\text{U}-} = \begin{bmatrix} 0.591  \\  0.214  \\  0.195 \end{bmatrix}, ~~
    w_{2}^{\text{L}+} = \begin{bmatrix} 0.333  \\  0.201  \\  0.195 \end{bmatrix}, ~~
    w_{2}^{\text{U}+} = \begin{bmatrix} 0.604  \\  0.333  \\  0.333 \end{bmatrix}
$"

...

L"$W = \begin{bmatrix}\left[ 0.472, 0.591 \right] \\ \left[ 0.214, 0.333 \right] \\ \left[ 0.195, 0.195 \right]\end{bmatrix}$"

L"$v^{\text{L}-} = \begin{bmatrix} 0.591  \\  0.214  \\  0.195 \end{bmatrix}, ~~
v^{\text{U}-} = \begin{bmatrix} 0.591  \\  0.214  \\  0.195 \end{bmatrix}, ~~
v^{\text{L}+} = \begin{bmatrix} 0.333  \\  0.201  \\  0.195 \end{bmatrix}, ~~
v^{\text{U}+} = \begin{bmatrix} 0.333  \\  0.201  \\  0.195 \end{bmatrix}
$"

L"$\varepsilon^\text{L} = \begin{bmatrix} 0.0  \\  0.0  \\  0.0 \end{bmatrix}, ~~
\varepsilon^\text{U} = \begin{bmatrix} 0.0  \\  0.0  \\  0.0 \end{bmatrix}
$"

L"$\text{結合後のPCM}$"

"\$\\hat{A} = \\begin{bmatrix} \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] & \\left[   1.0, \\left[ 2.202, 2.758 \\right], 2.935 \\right] & \\left[ 1.711, \\left[ 3.033, 3.033 \\right], 3.033 \\right] \\\\ \\left[ 0.341, \\left[ 0.363, 0.454 \\right],   1.0 \\right] & \\left[ " ⋯ 30 bytes ⋯ "ght],   1.0 \\right] & \\left[ 1.033, \\left[   1.1,   1.1 \\right], 1.711 \\right] \\\\ \\left[  0.33, \\left[  0.33,  0.33 \\right], 0.584 \\right] & \\left[ 0.584, \\left[ 0.909, 0.909 \\right], 0.968 \\right] & \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] \\end{bmatrix}\$"